# Bukti Endoginitas (Bisa dicoba di MetodeSimultan_executed.ipynb)

In [ ]:
import statsmodels.api as sm
import numpy as np

# ===== Durbin test untuk endogenitas Kemiskinan di persamaan IPM =====
# Pakai df_merged karena variabel utama + lag instrumen sudah ada di sini.

df_work = df_merged.copy()

# (opsional tapi disarankan) stabilkan skala PDRB
if 'log_pdrb_per_kapita' not in df_work.columns:
    df_work['log_pdrb_per_kapita'] = np.log(df_work['pdrb_per_kapita'].clip(lower=1))

# Pilih hanya kolom yang diperlukan & buang NA
need_cols = [
    'indeks_pembangunan_manusia',         # y
    'persentase_penduduk_miskin',         # xE (endogen)
    'rata_rata_lama_sekolah',             # X (eksogen)
    'log_pdrb_per_kapita',                # X (eksogen)
    'miskin_lag1', 'rls_lag1', 'pdrb_lag1'  # Z (instrumen)
]
dfw = df_work[need_cols].dropna()

# y (IPM), xE (Kemiskinan), X (eksogen), Z (instrumen)
y  = dfw['indeks_pembangunan_manusia'].to_numpy()
xE = dfw['persentase_penduduk_miskin'].to_numpy()
X  = dfw[['rata_rata_lama_sekolah','log_pdrb_per_kapita']].to_numpy()
Z  = dfw[['miskin_lag1','rls_lag1','pdrb_lag1']].to_numpy()

# Tahap 1: xE ~ Z + X  → ambil residual
X1 = sm.add_constant(np.column_stack([Z, X]))
fs = sm.OLS(xE, X1).fit()
u_hat = fs.resid

# Tahap 2 (Durbin): y ~ xE + X + u_hat
X2 = sm.add_constant(np.column_stack([xE, X, u_hat]))
durbin = sm.OLS(y, X2).fit(cov_type='HC1')

# Hasil ringkas (koefisien & p-value untuk u_hat kunci endogenitas)
print(durbin.summary().tables[1])  # lihat baris 'u_hat'
# Interpretasi: p-value u_hat < 0.05 → kemiskinan endogen → OLS bias.


## Tujuan Menjalankan Kode Durbin Test (Clarity)

**Apa yang diuji?**  
Mengukur apakah **variabel Kemiskinan** (sebagai penjelas di persamaan IPM) **endogen** atau tidak.

**Kenapa penting?**  
- Jika **endogen** → estimasi **OLS bias/tidak konsisten** → **harus** pakai **2SLS/IV**.  
- Jika **tidak endogen** → OLS **aman** untuk persamaan itu (2SLS jadi opsi robustness, bukan keharusan).

**Bagaimana caranya (intinya):**  
1. **Tahap 1**: prediksi Kemiskinan dengan **instrumen** (lag-lag) + eksogen → ambil **residual (`u_hat`)**.  
2. **Tahap 2**: regresikan IPM pada **Kemiskinan + eksogen + `u_hat`**.  
3. **Keputusan**:  
   - `p-value` **u_hat < 0,05** ⇒ **Endogen ada** ⇒ OLS bias ⇒ gunakan 2SLS.  
   - `p-value` **u_hat ≥ 0,05** ⇒ **Tidak ada bukti endogenitas** ⇒ OLS boleh dipakai.

**Apa yang barusan keluar di hasilmu?**  
- Baris **`x4` = `u_hat`** **tidak signifikan** (p = 0,139) → **tidak ada bukti kuat endogenitas** Kemiskinan pada spesifikasi yang dipakai.  
  ⇒ Untuk persamaan IPM ini, **OLS tidak terbukti bias** karena endogenitas Kemiskinan (pada 5%).

**Implikasi ke project:**
- Persamaan **IPM**: boleh laporkan **OLS** sebagai baseline **+** tampilkan **2SLS** sebagai robustness (karena model keseluruhan tetap simultan).  
- Persamaan **Kemiskinan**: **lakukan uji yang sama** (ganti variabel endogen ke IPM) untuk memastikan konsistensi keputusan.  
- Tetap cek: **kekuatan instrumen** (first-stage F > 10) & **validitas** (uji over-ID) agar 2SLS solid.

**Output yang diharapkan ditulis di laporan:**
- “Durbin test untuk endogenitas Kemiskinan di persamaan IPM menghasilkan p(u_hat)=0,139 → tidak signifikan.  
  Dengan demikian, pada spesifikasi ini tidak ditemukan bukti endogenitas. OLS dapat digunakan; 2SLS dilaporkan sebagai uji robustness.”

**Checklist singkat:**
- [ ] Jalankan Durbin test untuk **persamaan IPM** (done).  
- [ ] Jalankan Durbin test untuk **persamaan Kemiskinan** (uji IPM sebagai endogen).  
- [ ] Laporkan **first-stage F** (kekuatan instrumen).  
- [ ] (Jika over-identified) Laporkan **Hansen/Sargan** (validitas instrumen).  
- [ ] Ringkas keputusan: **kapan OLS dipakai, kapan 2SLS wajib**.
